In [1]:
%matplotlib inline
import matplotlib
from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, requests, pickle
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, Angle
from astropy.io import fits
from astropy import units as u
from style import output_folder, big_fontsize, base_width, base_height, dpi
import seaborn as sns
import json
from alerts import get_alerts
from astropy.time import Time

In [ ]:
alerts = get_alerts()
m = ~alerts.retracted
alert_coords = SkyCoord(alerts[m]['RA'], alerts[m]['Dec'], unit='deg')

cols = ['3FHL Name', '4FGL Name', 'Fermi ra', 'Fermi dec', 'Counterpart', 'Counterpart ra', 'Counterpart dec', 'Counterpart prob']
#fermi_matches = {k: list() for k in cols}
fermi_matches = pd.DataFrame(columns=cols)

for cat, fn in zip(['3FHL', '4FGL'], ["data/gll_psch_v13.fit", "data/gll_psc_v27.fit"]):

    fermi_3fhl = fits.open(fn)
    tab = fermi_3fhl[1]
    fermi_ra = tab.data["RAJ2000"]
    fermi_dec = tab.data["DEJ2000"]
    fermi_coords = SkyCoord(fermi_ra, fermi_dec, unit='deg')

    for j, (i, r) in enumerate(alerts[m].iterrows()):

        #if r.Event != 'IC200911A':
        #    continue

        c = SkyCoord(r['RA'], r['Dec'], unit='deg')
        delta_ra = fermi_coords.ra - c.ra
        delta_dec = fermi_coords.dec - c.dec

        try:    
            if r['Dec Unc (rectangle) float']:
                dec_unc = Angle(r['Dec Unc (rectangle) float']*u.deg)

                if r.Event == "IC190629A":
                    ra_unc = Angle([360, 360]*u.deg)
                else:
                    ra_unc = Angle(r['RA Unc (rectangle) float']*u.deg)

                match_mask = (delta_dec <= max(dec_unc)) & (delta_dec >= min(dec_unc)) & (delta_ra <= max(ra_unc)) & (delta_ra >= min(ra_unc))

            else:
                print(r)
                _sep = Angle(r["initial Error90 [arcmin]"] * u.arcmin)
                sep = c.separation(fermi_coords)
                match_mask = sep <= _sep

        except TypeError as e:
            print(r, "\n", e)

        #test_m = sep**2 == delta_dec**2 + delta_ra**2
        #if not np.all(test_m):
        #    raise Exception(r)

        _matches = tab.data[match_mask]
        
        if cat == '4FGL':
            matches = np.array([
                _matches["ASSOC_FHL"], 
                _matches['Source_Name'], 
                _matches['RAJ2000'], 
                _matches['DEJ2000'],
                _matches['ASSOC1'],
                _matches['RA_Counterpart'],
                _matches['DEC_Counterpart'],
                np.minimum(np.array(_matches['ASSOC_PROB_BAY']).astype(float), np.array(_matches['ASSOC_PROB_LR']).astype(float))
            ]).T
            columns = cols
            
        else:
            matches = np.array([
                _matches['Source_Name'], 
                _matches['RAJ2000'], 
                _matches['DEJ2000'],
                _matches['ASSOC1']
            ]).T
            columns = [f"{cat} Name", "Fermi ra", "Fermi dec", 'Counterpart']
        
        if len(_matches) > 0:
            print(f"{len(_matches)} matches in {cat} for {r.Event}")
            l = np.array([(r.Event, cat, j) for j in range(len(_matches))]).T
            index = pd.MultiIndex.from_arrays(l)
            ifermi_matches = pd.DataFrame(matches, columns=columns, index=index)
            fermi_matches = fermi_matches.append(ifermi_matches)
    
fermi_matches = pd.DataFrame(fermi_matches, 
                             index=pd.MultiIndex.from_tuples(fermi_matches.index,
                                                            names=['Event', 'Fermi Cat', 'Macth ind']))

### Math the 3FHL and the 4FGL Catalogues and remove the duplicat entries ###

In [5]:
drop_m = fermi_matches['3FHL Name'].duplicated(keep=False) & fermi_matches['4FGL Name'].isna()
print(f"found {len(fermi_matches[drop_m])} duplicates")
fermi_matches = fermi_matches[~drop_m]
print(len(fermi_matches))
print(len(fermi_matches[fermi_matches["4FGL Name"].isna() & (~fermi_matches["3FHL Name"].isna())]))

for x in ['ra', 'dec']:
    counterpart = fermi_matches[f"Counterpart {x}"]
    fermi = fermi_matches[f"Fermi {x}"]
    xm = (~counterpart.isna()) & (np.array(fermi_matches['Counterpart prob']).astype(float) > 0)
    fermi_matches[x] = fermi
    fermi_matches.loc[xm, x] = counterpart[xm]
    
has_cp = ~np.isnan(np.array(fermi_matches['Counterpart dec']).astype(float))

for k in ['Counterpart', 'Counterpart ra', 'Counterpart dec']:
    fermi_matches.loc[~has_cp, k] = ""

ncp = len(fermi_matches[has_cp])
print(f"{ncp} with counterpart, {len(fermi_matches) - ncp} without")
fermi_matches.to_csv('data/fermi_matches.csv')    
fermi_matches

found 55 duplicates
177
0
139 with counterpart, 38 without


3FHL Name           4FGL Name  \
Event     Fermi Cat Macth ind                                           
IC161103A 4FGL      0                              4FGL J0244.7+1316    
IC170922A 4FGL      0          3FHL J0509.4+0542   4FGL J0509.4+0542    
IC181014A 4FGL      0          3FHL J1457.5-3538   4FGL J1457.4-3539    
                    1                              4FGL J1505.0-3433    
IC181023A 4FGL      0                              4FGL J1804.4-0852    
...                                           ...                 ...   
IC210608A 4FGL      14                             4FGL J2248.9+2106    
IC210717A 4FGL      0                              4FGL J0256.2-0408    
                    1          3FHL J0304.5-0055   4FGL J0304.5-0054    
                    2                              4FGL J0307.8-0419    
IC210730A 4FGL      0                              4FGL J0659.7+1416    

                               Fermi ra Fermi dec  \
Event     Fermi Cat Macth ind                       
IC161103A 4FGL      0            41.192   13.2799   
IC170922A 4FGL      0           77.3593    5.7014   
IC181014A 4FGL      0          224.3657  -35.6527   
                    1          226.2581  -34.5546   
IC181023A 4FGL      0          271.1182   -8.8694   
...                                 ...       ...   
IC210608A 4FGL      14         342.2461   21.1159   
IC210717A 4FGL      0           44.0544    -4.134   
                    1           46.1423   -0.9148   
                    2           46.9523   -4.3266   
IC210730A 4FGL      0          104.9416   14.2792   

                                                Counterpart  \
Event     Fermi Cat Macth ind                                 
IC161103A 4FGL      0          GB6 J0244+1320                 
IC170922A 4FGL      0          TXS 0506+056                   
IC181014A 4FGL      0          PKS 1454-354                   
                    1          PMN J1505-3432                 
IC181023A 4FGL      0                                         
...                                                     ...   
IC210608A 4FGL      14         PKS 2246+208                   
IC210717A 4FGL      0                                         
                    1          RX J0304.5-0054                
                    2          LEDA 095522                    
IC210730A 4FGL      0          PSR J0659+1414                 

                                  Counterpart ra     Counterpart dec  \
Event     Fermi Cat Macth ind                                          
IC161103A 4FGL      0              41.1903889042          13.3353393   
IC170922A 4FGL      0              77.3581852833       5.69314818611   
IC181014A 4FGL      0               224.36129875      -35.6527698861   
                    1              226.259876379      -34.5491167778   
IC181023A 4FGL      0                                                  
...                                          ...                 ...   
IC210608A 4FGL      14             342.252361346          21.1174543   
IC210717A 4FGL      0                                                  
                    1              46.1414966667         -0.90130895   
                    2              46.9354854083      -4.31923459167   
IC210730A 4FGL      0          104.9505615234375  14.239310264587402   

                                 Counterpart prob                 ra  \
Event     Fermi Cat Macth ind                                          
IC161103A 4FGL      0                         0.0             41.192   
IC170922A 4FGL      0          0.9901848435401917      77.3581852833   
IC181014A 4FGL      0           0.994556725025177       224.36129875   
                    1          0.9717896580696106      226.259876379   
IC181023A 4FGL      0                         0.0           271.1182   
...                                           ...                ...   
IC210608A 4FGL      14         0.9745596051216125      342.25236134

In [ ]:
cmap = plt.get_cmap()

for i, r in alerts.iterrows():
    
    try:
        sources = fermi_matches.loc[pd.IndexSlice[r.Event, :, :]]
    except KeyError:
        continue
    
    sources = sources[~sources['4FGL Name'].isna()]
    
    if len(sources) > 0:
        print(sources.to_string(index=False))

        fig, ax = plt.subplots(figsize=(base_width, base_height), dpi=dpi)

        ax.set_aspect(1, adjustable='datalim')
        ax.scatter(r['RA'], r['Dec'], color=cmap(0), marker='s', s=2)
        if r['RA Unc (rectangle) float'] and r['Dec Unc (rectangle) float']:
            left_bottom_corner = (r['RA'] + r['RA Unc (rectangle) float'][1], r['Dec'] + r['Dec Unc (rectangle) float'][1])
            extent = (r['RA Unc (rectangle) float'][0] - r['RA Unc (rectangle) float'][1],
                      r['Dec Unc (rectangle) float'][0] - r['Dec Unc (rectangle) float'][1])
            final_e = plt.Rectangle(left_bottom_corner, extent[0], extent[1], fc=cmap(0), alpha=0.4, ec=None, ls='')
            ax.add_patch(final_e)
        else:
            print(r)

        ax.scatter(r['initial RA'], r['initial Dec'], color=cmap(0.2), s=2)
        initial_e = r['initial Error90 [arcmin]'] * u.arcmin
        initial_contour = plt.Circle((r['initial RA'], r['initial Dec']), initial_e.to('deg').value, alpha=0.4, fc=cmap(0.2), ec=None, ls='')
        ax.add_patch(initial_contour)

        ax.scatter(np.array(sources['ra']).astype(float), 
                   np.array(sources['dec']).astype(float), marker='x', color=cmap(0.8))
        fgl = ~sources['4FGL Name'].isna()
        for _, s in sources[fgl].iterrows():
            ylim = ax.get_ylim()
            yext = abs(ylim[1] - ylim[0])
            n = s['Counterpart'] if s['Counterpart'] else s['4FGL Name']
            ax.annotate(n, (float(s.ra), float(s.dec) + yext/12), ha='center', color=cmap(0.8), fontsize=6,
                       snap=True)

        ax.set_title(r.Event)
        major_base = 20 if r.Event=="IC190331A" in cc else 1
        ax.xaxis.set_major_locator(MultipleLocator(base=major_base))
        ax.yaxis.set_major_locator(MultipleLocator(base=major_base))
        ax.grid()

        ax.set_xlabel('RA')
        ax.set_ylabel('Dec')


        fig.tight_layout()
        fig.savefig(os.path.join(output_folder, f"{r.Event}.pdf"))
        try:
            plt.show()
        finally:
            plt.close()

In [8]:
pd.read_csv("data/fermi_matches.csv", index_col=[0,1,2])

3FHL Name           4FGL Name  \
Event     Fermi Cat Macth ind                                           
IC161103A 4FGL      0                              4FGL J0244.7+1316    
IC170922A 4FGL      0          3FHL J0509.4+0542   4FGL J0509.4+0542    
IC181014A 4FGL      0          3FHL J1457.5-3538   4FGL J1457.4-3539    
                    1                              4FGL J1505.0-3433    
IC181023A 4FGL      0                              4FGL J1804.4-0852    
...                                           ...                 ...   
IC210608A 4FGL      14                             4FGL J2248.9+2106    
IC210717A 4FGL      0                              4FGL J0256.2-0408    
                    1          3FHL J0304.5-0055   4FGL J0304.5-0054    
                    2                              4FGL J0307.8-0419    
IC210730A 4FGL      0                              4FGL J0659.7+1416    

                               Fermi ra  Fermi dec  \
Event     Fermi Cat Macth ind                        
IC161103A 4FGL      0           41.1920    13.2799   
IC170922A 4FGL      0           77.3593     5.7014   
IC181014A 4FGL      0          224.3657   -35.6527   
                    1          226.2581   -34.5546   
IC181023A 4FGL      0          271.1182    -8.8694   
...                                 ...        ...   
IC210608A 4FGL      14         342.2461    21.1159   
IC210717A 4FGL      0           44.0544    -4.1340   
                    1           46.1423    -0.9148   
                    2           46.9523    -4.3266   
IC210730A 4FGL      0          104.9416    14.2792   

                                                Counterpart  Counterpart ra  \
Event     Fermi Cat Macth ind                                                 
IC161103A 4FGL      0          GB6 J0244+1320                     41.190389   
IC170922A 4FGL      0          TXS 0506+056                       77.358185   
IC181014A 4FGL      0          PKS 1454-354                      224.361299   
                    1          PMN J1505-3432                    226.259876   
IC181023A 4FGL      0                                   NaN             NaN   
...                                                     ...             ...   
IC210608A 4FGL      14         PKS 2246+208                      342.252361   
IC210717A 4FGL      0                                   NaN             NaN   
                    1          RX J0304.5-0054                    46.141497   
                    2          LEDA 095522                        46.935485   
IC210730A 4FGL      0          PSR J0659+1414                    104.950562   

                               Counterpart dec  Counterpart prob          ra  \
Event     Fermi Cat Macth ind                                                  
IC161103A 4FGL      0                13.335339          0.000000   41.192000   
IC170922A 4FGL      0                 5.693148          0.990185   77.358185   
IC181014A 4FGL      0               -35.652770          0.994557  224.361299   
                    1               -34.549117          0.971790  226.259876   
IC181023A 4FGL      0                      NaN          0.000000  271.118200   
...                                        ...               ...         ...   
IC210608A 4FGL      14               21.117454          0.974560  342.252361   
IC210717A 4FGL      0                      NaN          0.000000   44.054400   
                    1                -0.901309          0.938326   46.141497   
                    2                -4.319235          0.835451   46.935485   
IC210730A 4FGL      0                14.239310          0.987233  104.950562   

                                     dec  
Event     Fermi Cat Macth ind             
IC161103A 4FGL      0          13.279900  
IC170922A 4FGL      0           5.693148  
IC181014A 4FGL      0         -35.652770  
                    1         -34.549117  
IC181023A 4FGL      0          -8.869400  
...              